In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
#Load data
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [9]:
#Cleaning data
train_data.drop_duplicates()
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].median())
train_data["Embarked"] = train_data["Embarked"].fillna(train_data["Embarked"].mode()[0])
train_data["Sex"] = train_data["Sex"].replace({"male":0, "female":1})
train_data["Embarked"] = train_data["Embarked"].replace({"S":0, "C":1, "Q":2})
train_data["FamilySize"] = train_data["SibSp"] + train_data["Parch"]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,FamilySize
0,0,3,0,22.0,1,0,7.2500,NaN,0,1
1,1,1,1,38.0,1,0,71.2833,C85,1,1
2,1,3,1,26.0,0,0,7.9250,NaN,0,0
3,1,1,1,35.0,1,0,53.1000,C123,0,1
4,0,3,0,35.0,0,0,8.0500,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...
885,0,3,1,39.0,0,5,29.1250,NaN,2,5
887,1,1,1,19.0,0,0,30.0000,B42,0,0
888,0,3,1,28.0,1,2,23.4500,NaN,0,3
889,1,1,0,26.0,0,0,30.0000,C148,1,0


In [13]:
train_data.to_csv("cleaned_titanic.csv", index=False)

In [ ]:
df = pd.read_csv("cleaned_titanic.csv")

In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

In [ ]:
age_alive_men = train_data.loc[(train_data.Survived) & (train_data.Sex == 'male'), "Age"]
age_avg_alive_men = age_alive_men.mean()
print("Average age of survivors who were men: ",age_avg_alive_men )

age_alive_women = train_data.loc[(train_data.Survived) & (train_data.Sex == 'female'), "Age"]
age_avg_alive_women = age_alive_women.mean()
print("Average age of survivors who were women: ",age_avg_alive_women )

age_dead_men = train_data.loc[(train_data.Survived == 0) & (train_data.Sex == 'male'), "Age"]
age_avg_dead_men = age_dead_men.mean()
print("Average age of dead men: ",age_avg_dead_men )

age_dead_women = train_data.loc[(train_data.Survived == 0) & (train_data.Sex == 'female'), "Age"]
age_avg_dead_women = age_dead_women.mean()
print("Average age of dead women: ",age_avg_dead_women )

In [ ]:
avg_paid = train_data.Fare.mean()
print("Average Fare: ",avg_paid)
avg_paid_alive = train_data.loc[train_data.Survived, "Fare"].mean()
print("Average Fare for survivors:",avg_paid_alive)
avg_paid_dead = train_data.loc[train_data.Survived==0, "Fare"].mean()
print("Average Fare for dead:", avg_paid_dead)

In [ ]:
embarked_count_survive_S = ((train_data.Survived)&(train_data["Embarked"]=="S")).sum()
embarked_count_dead_S = ((train_data.Survived==0)&(train_data["Embarked"]=="S")).sum()
print("Embark in S and live:",embarked_count_survive_S)
print("Embark in S and die:",embarked_count_dead_S)
embarked_count_survive_C = ((train_data.Survived)&(train_data["Embarked"]=="C")).sum()
embarked_count_dead_C = ((train_data.Survived==0)&(train_data["Embarked"]=="C")).sum()
print("Embark in C and live:",embarked_count_survive_C)
print("Embark in C and die:",embarked_count_dead_C)



In [ ]:
from sklearn.ensemble import RandomForestClassifier

test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators = 100, max_depth = 5, random_state = 1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId' : test_data.PassengerId, 'Survived': predictions})
output.to_csv('submissions.csv', index=False)
print("Your submission was saved")